# Follow Road Practice
<img src="images/jderobot.png" width="15%" height="15%" style="float:left;padding-right:15px"/>

## 1- Introduction
---

In this exercise we are going to implement a "Formula 1" intelligence to follow a red line across the circuit. To do it, the student needs to have at least the next knowledge:
* Python programming skills
* Color spaces (RGB, HSV, etc)
* Basic understanding of [OpenCV library](http://opencv.org/)

## 2- Exercise components

<img src="images/drone.png" width="30%" height="30%" style="float:right;padding-right:15px"/>
### 2.1- Gazebo simulator
---
Gazebo simulator will be running in the background. The Gazebo world employed for this exercise has one element: a simulated drone.The drone robot will provide camera where the images will be provided to the student.

### 2.2 Follow Road Component
This component has been developed specifically to carry out this exercise. This component connects to Gazebo to teleoperate the drone (or send orders to it) and receives images from its camera. The student has to modify this component and add code to accomplish the exercise. In particular, it is required to modify the execute() method.

## 3- Exercise initialization
---
First of all, we need to run the Gazebo simulator:

In [ ]:
import subprocess
gazebo = subprocess.Popen(("gazebo", "road_drone_textures.world"))

After launch the gazebo simulator, we must wait a few second to let the gazebo's world be opened. Then a world like the one shows in the following picture should appears:

<img src="images/world.png" width="50%" height="50%">

Now, we must import the code with the functions of the practice

In [ ]:
import threading
import time
from datetime import datetime
import math
import jderobot
import cv2
import numpy as np

time_cycle = 80
hmin = 20
smin= 0
vmin= 0
hmax= 60
smax= 130
vmax= 130

class PID:
    """
	Discrete PID control
	"""

    def __init__(self, P=2.0, I=0.0, D=1.0, Derivator=0, Integrator=0, Integrator_max=500, Integrator_min=-500):

        self.Kp = P
        self.Ki = I
        self.Kd = D
        self.Derivator = Derivator
        self.Integrator = Integrator
        self.Integrator_max = Integrator_max
        self.Integrator_min = Integrator_min

        self.set_point = 0.0
        self.error = 0.0

    def update(self, current_value):
        """
		Calculate PID output value for given reference input and feedback
		"""

        self.error = self.set_point - current_value

        self.P_value = self.Kp * self.error
        self.D_value = self.Kd * ( self.error - self.Derivator)
        self.Derivator = self.error

        self.Integrator = self.Integrator + self.error

        if self.Integrator > self.Integrator_max:
            self.Integrator = self.Integrator_max
        elif self.Integrator < self.Integrator_min:
            self.Integrator = self.Integrator_min

        self.I_value = self.Integrator * self.Ki

        PID = self.P_value + self.I_value + self.D_value

        return PID

    def setPoint(self, set_point):
        """
		Initilize the setpoint of PID
		"""
        self.set_point = set_point
        self.Integrator = 0
        self.Derivator = 0

    def setIntegrator(self, Integrator):
        self.Integrator = Integrator

    def setDerivator(self, Derivator):
        self.Derivator = Derivator

    def setKp(self, P):
        self.Kp = P

    def setKi(self, I):
        self.Ki = I

    def setKd(self, D):
        self.Kd = D

    def getPoint(self):
        return self.set_point

    def getError(self):
        return self.error

    def getIntegrator(self):
        return self.Integrator

    def getDerivator(self):
        return self.Derivator


class MyAlgorithm(threading.Thread):

    def __init__(self, camera, navdata, pose, cmdvel, extra):
        self.camera = camera
        self.navdata = navdata
        self.pose = pose
        self.cmdvel = cmdvel
        self.extra = extra
    
        self.stop_event = threading.Event()
        self.stop_event.set()
        self.kill_event = threading.Event()
        
        self.threshold_image = np.zeros((640,360,3), np.uint8)
        self.color_image = np.zeros((640,360,3), np.uint8)
        
        self.lock = threading.Lock()
        
        self.threshold_image_lock = threading.Lock()
        self.color_image_lock = threading.Lock()
        threading.Thread.__init__(self, args=self.stop_event)
        
    def getImage(self):
        self.lock.acquire()
        img = self.camera.getImage().data
        self.lock.release()
        return img

    def set_color_image (self, image):
        img  = np.copy(image)
        if len(img.shape) == 2:
          img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        self.color_image_lock.acquire()
        self.color_image = img
        self.color_image_lock.release()
        
    def get_color_image (self):
        self.color_image_lock.acquire()
        img = np.copy(self.color_image)
        self.color_image_lock.release()
        return img
        
    def set_threshold_image (self, image):
        img = np.copy(image)
        if len(img.shape) == 2:
          img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        self.threshold_image_lock.acquire()
        self.threshold_image = img
        self.threshold_image_lock.release()
        
    def get_threshold_image (self):
        self.threshold_image_lock.acquire()
        img  = np.copy(self.threshold_image)
        self.threshold_image_lock.release()
        return img
    
    def run (self):

        self.stop_event.clear()

        while (not self.kill_event.is_set()):
           
            start_time = datetime.now()

            if not self.stop_event.is_set():
                self.algorithm()

            finish_Time = datetime.now()

            dt = finish_Time - start_time
            ms = (dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
            #print (ms)
            if (ms < time_cycle):
                time.sleep((time_cycle - ms) / 1000.0)

    def stop (self):
        self.stop_event.set()

    def play (self):
        if self.is_alive():
            self.stop_event.clear()
        else:
            self.start()

    def kill (self):
        self.kill_event.set()
        
    def reset(self):
        self.xPid = PID()
        self.yPid = PID()
        
    def algorithm(self):
        # Add your code here
        pass
    
from IPython.display import HTML
from IPython.display import display

# Script to hide the code
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

To start coding, we need to call Follow_Line class once. Run this code and wait a few seconds until follow line initialization finishes with an OK message

In [ ]:
import sys
import comm
import config
from MyAlgorithm import MyAlgorithm, PID
sys.path.append('/usr/lib/python2.7/')
import types

import signal

signal.signal(signal.SIGINT, signal.SIG_DFL)

class FollowRoad():
    
    def __init__(self):
        print "Initializing"
        cfg = config.load("follow_road_conf.yml")

        #starting comm
        jdrc= comm.init(cfg, 'Introrob')

        self.camera = jdrc.getCameraClient("Introrob.Camera")
        self.navdata = jdrc.getNavdataClient("Introrob.Navdata")
        self.pose = jdrc.getPose3dClient("Introrob.Pose3D")
        self.cmdvel = jdrc.getCMDVelClient("Introrob.CMDVel")
        self.extra = jdrc.getArDroneExtraClient("Introrob.Extra")  

        self.algorithm = MyAlgorithm(self.camera, self.navdata, self.pose, self.cmdvel, self.extra)
        print "Follow_Road Components initialized OK"
    
    def play(self):
        # self.unpause_physics_client()
        self.algorithm.play()
        print "Follow_Road is running"
        
    def pause (self):
        self.algorithm.pause()
        #self.pause_physics_client()
        print "Follow_Road has been paused"
         
    def stop(self):
        self.algorithm.stop()
        print "Follow_Road stopped"
                   
    def setExecute(self, execute):
        self.algorithm.algorithm = types.MethodType(execute, self.algorithm )
        print "Code updated"       

    def move(self):
        self.algorithm.move()
    
    def get_threshold_image(self):
        return self.algorithm.get_threshold_image()
    
    def get_color_image(self):
        return self.algorithm.get_color_image()
    
%matplotlib inline

fr = FollowRoad()
fr.play()

from IPython.display import HTML
from IPython.display import display

# Script to hide the code
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

Remember to send the takeoff order to the drone, so you can see anything on its camera:

In [ ]:
fr.extra.takeoff()  #We will go over the drone's API in step 3.1

Now we can start coding to give intelligence to the Formula1 robot. We can do it modifying the execute() method from Follow Road component. This method will be called iteratively. Each iteration, we'll print a message.

In [ ]:
def execute(self):
    # Add your code here
    input_image = self.camera.getImage().data

    if input_image is not None:
        image_HSV = cv2.cvtColor(input_image, cv2.COLOR_RGB2HSV)

        #Treshold image
        value_min_HSV = np.array([20, 0, 0])
        value_max_HSV = np.array([60, 130, 130])
        image_HSV_filtered = cv2.inRange(image_HSV, value_min_HSV, value_max_HSV)

        #Reducing noise
        closing = cv2.morphologyEx(image_HSV_filtered, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))

        #Filtered image
        image_HSV_filtered_Mask = np.dstack((opening, opening, opening))

        #drawing contours
        imgray = cv2.cvtColor(image_HSV_filtered_Mask, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, 255, 0)
        _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(image_HSV_filtered_Mask, contours, -1, (0,255,0), 3)

        #Getting the centre of the road
        area = []
        for pic, contour in enumerate(contours):
            area.append(cv2.contourArea(contour))
        if len(area) > 1:
            if area[0] < area[1]:
                M = cv2.moments(contours[1])
            else:
                M = cv2.moments(contours[0])
        else:
            M = cv2.moments(contours[0])

        if int(M['m00']) != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            print("cx: " + str(cx))
            print("cy: " + str(cy))

            #drawing the center
            cv2.circle(image_HSV_filtered_Mask, (cx, cy), 7, np.array([255, 0, 0]), -1)

            #move the drone
            if cx > 250:
                self.cmdvel.sendCMDVel(0,0,0.5,0,0,0)
                print("Elevating")
            elif cx < 20:
                self.cmdvel.setVX(0.3)
                self.cmdvel.setYaw(-0.1)
                print("Detected two roads")

            else:
                self.cmdvel.setVX(0.3)
                self.cmdvel.setYaw((153-int(cx))*0.01)

                print("Yaw: " + str((153-int(cx))*0.01))
            self.cmdvel.sendVelocities()

        #printing the filtered image
        self.set_threshold_image(image_HSV_filtered_Mask)
        
fr.setExecute(execute)

In [ ]:
# @hidden_cell
# Implement execute method
def execute(self):
    print "Running execute iteration"
      
fr.setExecute(execute)

Stop printing the updating of the method with an empty instruction:

In [ ]:
def execute(self):
    pass

fr.setExecute(execute)

### 3.1 - API
---
 - Te way to send orders to the robot is the following:
```
self.cmdvel.sendCMDVel(valueX,valueY,0,0,0,0)
```

- You can also get navigation data with:
```
data = self.navdata.getNavData() 
```
This will give you state (data.state), altitude (data.altd), vehicle (data.vehicle) and battery (data.batteryPercent).

- Get the image from the camera as:
```
img = self.getImage()
```
- Finally, you can use the drone's API, that allows you to:
```
self.pose.getPose3d().x, self.pose.getPose3d().y, self.pose.getPose3d().z
```
returns the position values of the drone in space.
```
self.pose.getPose3d().roll, self.pose.getPose3d().pitch, self.pose.getPose3d().yaw
```
returns the rotation values of the drone in space.
```
self.extra.takeOff()
```
Takeoff of the drone.
```
self.extra.land()
```
landing of the drone.


To get images from the camera, you must import the printer function:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def printImage(image):
    plt.axis('off')
    a = plt.imshow(image)

To save the images, you can use the following commands:

In [ ]:
def execute(self):
    img = self.getImage()
    self.set_color_image(img)
    
fr.setExecute(execute)

When you have the image saved, you can use these instructions to show the camera images:

In [ ]:
imageCamera = fr.get_color_image()
printImage(imageCamera)

Or you can use the following instructions to show the filtered images:

In [ ]:
def execute(self):
    img = self.getImage()
    self.set_threshold_image(img)
    
fr.setExecute(execute)

In [ ]:
segmentedImage = fr.get_threshold_image()
printImage(segmentedImage)